<a href="https://colab.research.google.com/github/drbarak/NLP-FinalProject/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# if fails to send open google 'unlock captcha' and activate it https://accounts.google.com/b/0/DisplayUnlockCaptcha
EMAIL = False
if EMAIL:
  import smtplib
  p = print

  from email.message import EmailMessage

  msg = EmailMessage()
  msg['Subject'] = 'New user'
  msg['From'] = "Weatherbot <dr.zvibarak@gmail.com>"
  msg['To'] = "drbarak@talkie.co.il"
  msg.set_content('This is my message')

  try:
    smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpObj.starttls()
    smtpObj.login('dr.zvibarak@gmail.com', 'shushu1952')
    smtpObj.send_message(msg)
    smtpObj.quit()         
    print("Successfully sent email")
  except Exception as e:# SMTPException:
    print("Error: unable to send email")
    p(e)


SMTPAuthenticationError: (534, b'5.7.14 <https://accounts.google.com/signin/continue?sarp=1&scc=1&plt=AKgnsbu\n5.7.14 wikqxaraOBaoJ8uaTDqCVoXmzPp36d_A6D8Ay2dQ-cRIGSVS1O88IW5d5hVoC21k64yZ8\n5.7.14 zt8t4BLQx1zO_S0arOXBdjPX4pd3KRLEhWoYyXJorkcAkitmYm1SAhepjia4Eep4>\n5.7.14 Please log in via your web browser and then try again.\n5.7.14  Learn more at\n5.7.14  https://support.google.com/mail/answer/78754 m13sm2830102vsl.16 - gsmtp')

In [2]:
INSTALL = False
if INSTALL:
  # to switch version of spacy
  !pip uninstall spacy
  !pip uninstall en_core_web_mdy
  
  !pip install spacy==2.2.4 #3.0.6

  !pip install pyforest
  #!pip install wget
  !python -m spacy download en_core_web_md

# INIT

In [3]:
from importlib import import_module
p = print
def my_import(lib, package=None, func=None, version=None):
  try:
    import_module(lib)
  except:
    if package is None: package = lib
    if version:
      p('pip install "{package}=={version}"')
      !pip install "{package}=={version}"
    else:
      p('pip install "{package}"')
      !pip install "{package}"
    import_module(lib)
  
  if func is not None:
    return import_module(lib).__getattribute__(func)

In [ ]:
my_import('pyforest')
import json, bs4
import string
import random
import time

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
space_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)

p = print
d = display

In [ ]:
#my_import('spacy', version='3.0.6')
p('spacy.__version__', spacy.__version__)
from spacy.tokens import Doc, Span, Token
from spacy.matcher import PhraseMatcher
from spacy.language import Language

# Load md (medium) English tokenizer, tagger, parser and NER, and vectors(vectors are not included in the 'sm' model)
#nlp = spacy.load('en_core_web_md')  NOT WORKING ON COLAB
try:
  import en_core_web_md
except:  
  !python -m spacy download en_core_web_md
  import en_core_web_md
  
nlp = en_core_web_md.load()

In [ ]:
#!pip uninstall googletrans
my_import('googletrans', version='3.1.0a0')
from googletrans import Translator
translator = Translator()

In [78]:
import requests

api_key = 'c2adfa29edfd95ad16efab9218619ff3'
URL = "http://api.openweathermap.org/data/2.5/{0}?"
icon_url = " http://openweathermap.org/img/wn/{0}@2x.png"  # 10d
#URL_cnt = "http://api.openweathermap.org/data/2.5/forecast/daily?q={city name}&cnt={cnt}&appid={API key}
#URL_cnt = "//https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=minutely, hourly&appid={API key}



In [9]:
path = '/content/drive/MyDrive/Project5_ChatBot/'
data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/' 

In [10]:
TEST = True
LANG = 'en'

In [11]:
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:   # len(nlp.Defaults.stop_words)=326
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            p('-PRON-: ', token.lemma_)
            continue
        result.append(token.lemma_)
    return " ".join(result)

In [12]:
def calculate_similarity(text1, text2):
    def get_similarity(doc1, doc2):
      if doc1 and doc1.vector_norm:
        if doc2 and doc2.vector_norm:
          return doc1.similarity(doc2)
      return 0

    doc1 = nlp(text1)
    #p(doc1.pos_)
    doc2 = nlp(text2)
    similarity1 = get_similarity(doc1, doc2)

    doc1 = nlp(process_text(text1))
    #p(doc1)
    doc2 = nlp(process_text(text2))
    #p(doc2)
    similarity2 = get_similarity(doc1, doc2)

    return max(similarity1, similarity2)

# Get cities and update model

In [13]:
GIT = True
git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/'

def get_json(fname, GIT=True):
  p(fname)
  if GIT:
    resp = requests.get(data_url + fname)
    return json.loads(resp.text)
  with open(path + fname, encoding="utf8") as f:
    return json.loads(f.read())

In [84]:
my_import('wget')
import wget

def load_CITIES_API():
  # file too big for git so read it from my google drive using a shared link
  CITIES_API = get_json('city.list.json')
  ''' 
  git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/' + 'city.list.json'
  resp = requests.get(git_path)
  CITIES_API = json.loads(resp.text)

  google_shared_link = 'https://drive.google.com/file/d/19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT/view?usp=sharing'
  url = 'https://drive.google.com/uc?authuser=0&id=19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT&export=download'
  file_id = '19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT'
  
  while True:
    try:
      with open('city.list.json', encoding="utf8") as f:
        CITIES_API = json.loads(f.read())
      break
    except:
      wget.download(url)  # download the file to local google drive with the original name in the share link
      # now can open with no path because it is in the local 'colab' directory
  
  '''
  df = pd.DataFrame(CITIES_API)
  df = df.drop(['id'], axis='columns')
  df.name = df.name.str.lower()
  df.name = df.name.str.translate(remove_punct_dict)

  CITIES_API_city = sorted(set(df.name.to_list()))
  CITIES_API_country = sorted(set(df.country.str.lower().to_list()))
  if CITIES_API_country[0] == '':
    CITIES_API_country = CITIES_API_country[1:]
  CITIES_API_state = sorted(set(df.state.str.lower().to_list()))
  if CITIES_API_state[0] == '':
    CITIES_API_state = CITIES_API_state[1:]
  if CITIES_API_state[0] == '00':
    CITIES_API_state = CITIES_API_state[1:]
  
  df.set_index('name', inplace=True)
  #d(df.head)
  return df, CITIES_API_city, CITIES_API_country, CITIES_API_state

df_CITIES_API, CITIES_API_city, CITIES_API_country_code, CITIES_API_state_code = load_CITIES_API()
df_CITIES_API.head()
#df_CITIES_API.loc['taglag'].coord

city.list.json


<IPython.core.display.Javascript object>

,state,country,coord
name,,,
ḩeşāre sefīd,,IR,"{'lon': 47.159401, 'lat': 34.330502}"
‘ayn ḩalāqīm,,SY,"{'lon': 36.321911, 'lat': 34.940079}"
taglag,,IR,"{'lon': 44.98333, 'lat': 38.450001}"
qabāghlū,,IR,"{'lon': 46.168499, 'lat': 36.173302}"
‘arīqah,,SY,"{'lon': 36.48336, 'lat': 32.889809}"


In [15]:
def load_countries():
  path_ = path  
  fname = 'countries.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname)
  df.Name = df.Name.str.lower()
  COUNTRIES = df.Name.tolist()
  df = df.set_index('Code')
  return df, COUNTRIES

countries_df, COUNTRIES = load_countries()
countries_df.head()

<IPython.core.display.Javascript object>

,Name
Code,
AF,afghanistan
AX,åland islands
AL,albania
DZ,algeria
AS,american samoa


In [16]:
countries_df[countries_df.Name == 'israel'].iloc[0].name
countries_df.loc['US']

Name    united states
Name: US, dtype: object

In [17]:
def load_cities_il():
  path_ = path
  fname = 'cities_IL.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, names=['city', 'district'])
  df.city = df.city.str.translate(remove_punct_dict)
  CITIES = df.city.tolist()
  df = df.set_index('city')
  return df, CITIES
cities_il_df, CITIES_IL = load_cities_il()
cities_il_df.head()

<IPython.core.display.Javascript object>

,district
city,
asam,beersheba
abbirim,acre
abu abdun,beersheba
abu ammar,beersheba
abu amre,beersheba


In [18]:
def load_capitals():
  path_ = path
  fname = 'capitals.json'
  if GIT:
    path_ = git_path
  CAPITALS = pd.read_json(path_ + fname, encoding="utf8", typ='series').str.lower()

  CAPITALS.index = CAPITALS.index.str.lower()
  df = pd.DataFrame([CAPITALS])
  df = df.T
  df = df.reset_index()
  df.columns = ['country', 'capital']
  df = df.set_index('capital')
  return df, CAPITALS.to_dict()
capitals_df, CAPITALS = load_capitals()

p(CAPITALS['albania'])
p(capitals_df.loc['madrid'].country)
capitals_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

tirana
spain


,country
capital,
kabul,afghanistan
mariehamn,åland islands
tirana,albania
algiers,algeria
pago pago,american samoa


In [19]:
def load_largest():
  fname = '100-largest-cities.csv'
  path_ = path
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, encoding='ISO-8859-8')
  df.city = df.city.str.lower()
  df.city = df.city.str.translate(remove_punct_dict)
  LARGEST = df.city.tolist()
  df = df.set_index('city')
  return df, LARGEST
largest_df, LARGEST = load_largest()

d(largest_df.head())
p(largest_df.loc['toronto'].country)

<IPython.core.display.Javascript object>

,country
city,
tokyo,Japan
delhi,India
shanghai,China
so paulo,Brazil
ciudad de mxico mexico city,Mexico


Canada


In [51]:
def load_actions():
  path_ = path
  fname = 'actions.json'
  actions = get_json(fname, False)
  ACTIONS_patterns_ = [y for tag in actions["actions"] for y in tag['patterns']]
  ACTIONS_ = {y: tag['tag'] for tag in actions["actions"] for y in tag['patterns']}
  ACTIONS_tags_ = {tag['tag']:'' for tag in actions["actions"]}
  return ACTIONS_patterns_, ACTIONS_, ACTIONS_tags_
ACTIONS_patterns, ACTIONS, ACTIONS_tags = load_actions()

p(ACTIONS_patterns[-5:])
p(list(ACTIONS.items())[:5])
p(ACTIONS.keys())
p(ACTIONS_tags)
ACTIONS['hot']
ACTIONS_tags.keys()
'clouds' in ACTIONS

actions.json
['evening', 'sundown', 'cloud', 'clouds', 'fahrenheit']
[('weather', 'all'), ('tornedo', 'weatherWind'), ('hurricane', 'weatherWind'), ('gale', 'weatherWind'), ('gust', 'weatherWind')]
dict_keys(['weather', 'tornedo', 'hurricane', 'gale', 'gust', 'whirlwind', 'cyclone', 'fog', 'mist', 'isobar', 'barometric', 'heat wave', 'freeze', 'climate', 'conditions', 'forecast', 'outlook', 'thunder', 'thunderstorm', 'storm', 'rain', 'drizzle', 'snow', 'dry', 'rainbow', 'overcast', 'shower', 'lightning', 'blustery', 'downpour', 'tropical', 'water cycle', 'drought', 'hail', 'icicle', 'flood', 'muggy', 'flash flood', 'atmospheric', 'cold front', 'cold snap', 'condensation', 'ice storm', 'snowfall', 'air', 'balmy', 'avalanche', 'frost', 'blizzard', 'spring', 'smog', 'ozone', 'autumn', 'winter', 'summer', 'sleet', 'sky', 'easterlies', 'westerlies', 'dew', 'surge', 'monsoon', 'permafrost', 'temperature', 'hot', 'warn', 'cold', 'temp', 'degree', 'thermal', 'heat', 'heat index', 'warm', 'pres

True

## Messages

In [21]:
def load_messages():
  path_ = path
  fname = 'messages.json'
  messages = get_json(fname, False)
  intents = {lang: messages["intents"]["language"][lang] for lang in messages["intents"]["language"]}
  #MESSAGES_lang = [lang for lang in messages["messages"]["language"]]
  MESSAGES = {lang: messages["messages"]["language"][lang] for lang in messages["messages"]["language"]}
  MESSAGES_lang = list(MESSAGES.keys())
  MESSAGES_en, intents_en = [], []
  if 'en' in MESSAGES:
    MESSAGES_en = MESSAGES['en']["messages"]
    for part in intents['en']["messages"]:
      intents_en.append(part["responses"])
  # read also the new_mssages
  
  try:
    with open(path + 'new_messages.json', "r") as f:
      list_d = json.loads(f.read())
  except Exception as error:
    list_d = []
  if len(list_d) > 0:
    for dic in list_d:
      intents.update(dic['intents']['language'])
      MESSAGES.update(dic['messages']['language'])
    MESSAGES_lang = list(MESSAGES.keys())
  return MESSAGES_lang, MESSAGES, MESSAGES_en, intents, intents_en
MESSAGES_lang, MESSAGES, MESSAGES_en, intents, intents_en = load_messages()
p(MESSAGES_lang)
p(intents_en)
#intents['he']["messages"][0]
intents_en[1]

messages.json
['en', 'he', 'fr', 'es', 'ar', 'ru']
[["Hi. My name is WeatherBot and I'm a chatbot. If you want to exit, type 'Bye' If you want help type 'Help'"], ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], ['Happy to help!', 'Any time!', 'My pleasure'], ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], ['I can give you information about the weather in cities around the world, in your language. You can ask about clouds, rain, temp, pressure, humidity, wind, visibility, sunrise and upto 16 days from today'], ['WeatherBot', 'Ask Bot', 'Debug', 'User Input', 'You', 'Bot', 'ID', 'Conversation']]


['See you!', 'Have a nice day', 'Bye! Come back again soon.']

In [22]:
def set_capitals():
  # Getter that looks up the span text in the dictionary of country capitals
  get_capital = lambda span: CAPITALS.get(span.text)

  # Register the Span extension attribute "capital" with the getter get_capital
  Span.set_extension("capital", getter=get_capital, force=True)

  # Process the text and print the entity text, label and capital attributes
  doc = nlp("Czech Republic may help Slovakia protect its airspace")
  print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

# Call the web site

{"coord":{"lon":34.8,"lat":32.0833},

"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02n"}],

"base":"stations",

"main":{"temp":295.69,"feels_like":295.83,"temp_min":294.42,"temp_max":296.58,"pressure":1016,"humidity":70},

"visibility":10000,

"wind":{"speed":2.06,"deg":10},

"clouds":{"all":20},

"dt":1623961727,

"sys":{"type":1,"id":6845,"country":"IL","sunrise":1623897263,"sunset":1623948546},

"timezone":10800,

"id":293396,

"name":"Tel Aviv",

"cod":200}

In [206]:
def call_web(city_name, country, fahrenheit, count=1):
    state = ''
    api_url = f"{URL}q={city_name},{state},{country}&units={'metric' if not fahrenheit else 'imperial'}&lang={LANG}&appid={api_key}".format('weather')
    if count > 1:
      cor = df_CITIES_API.loc[city_name].coord
      #p(city_name, cor)
      api_url = f"{URL}lat={cor['lat']}&lon={cor['lon']}&exclude=minutely,hourly&units={'metric' if not fahrenheit else 'imperial'}&lang={LANG}&appid={api_key}".format('onecall')

    #p(api_url)

    response = requests.get(api_url)
    response_dict = response.json()

    weather = response_dict #["weather"]#[0]#["description"]

    if response.status_code == 200:
        return weather
    elif response.status_code == 404:
        return None
    else:
        print(MESSAGES[LANG]["messages"]["web_error"].format(response.status_code, api_url))
        p(response_dict['message'])
        return None

In [220]:
def get_weather_info(actions, count, cnt, city_weather, default, LANG, DISP):
  all_action = actions[0][0] == 'all'  # always first
  #p(all_action)
  result = ''
  for action in actions:
    tag = action[0]
    if tag == "fahrenheit": continue
    user_act = action[1]
    if count < 2:
      data = city_weather if tag == "all" else city_weather['main'] \
        if tag in ["weatherTemp", "weatherPressure", 'temp', "pressure", "humidity"] else \
              city_weather['sys'] if tag == "sunrise" else city_weather['wind'] \
              if tag in ["weatherWind", "wind"] else city_weather[tag] 
    else:
      data = city_weather
    if DISP: p('in loop', user_act, tag, data)
    if tag in ['clouds']:
      clouds_percentage = data['all'] if count < 2 else data['clouds']
      if clouds_percentage < 10: # no clouds
        if 'clear sky' not in default.lower():
          result += f'{MESSAGES[LANG]["messages"]["clear sky"]}, '
        elif not all_action:
          result += f' {default},'
      elif default not in result:
        result += f' {default},'
      if all_action: continue
    if user_act == "rain": 
      if user_act in default.lower():
        if default not in result:
          result += f' {default},'
      else:
        result += f' {MESSAGES[LANG]["messages"]["no"]} {user_act},'
      if all_action: continue
    if all_action and tag != 'all': continue
    if tag in ["weather", "weatherPressure","weatherWind","weatherTemp","all"]:
      if default not in result:
        result += f' {default},'
    if tag in ["weatherTemp", "weatherPressure", 'temp', "pressure", "humidity", "all"]:
      data_tag = data['main'] if tag == 'all' and count < 2 else data
      #p(count, tag, data_tag, data)
      if tag in [ "weatherTemp", "temp", "all"]:
        for i, temp in enumerate(MESSAGES[LANG]["messages"]["temp"]):
          if i == 0: 
            if cnt == 0:
              info = data_tag["temp"]
            else:
              info = data_tag["temp"]['day']
          elif i == 1: 
            if cnt == 0:
              info = data_tag["feels_like"]
            else:
              info = data_tag["feels_like"]['day']
            if count > 1  and cnt == 0:
              break
          elif i == 2: info = data_tag["temp_min"] if cnt == 0 else data_tag["temp"]["min"]
          elif i == 3: info = data_tag["temp_max"] if cnt == 0 else data_tag["temp"]["max"]
          result += f' {temp} = {info},'
      if tag in ["weatherPressure", "pressure", "all"]:
        result += f' {MESSAGES[LANG]["messages"]["pressure"]} = {data_tag["pressure"]},'
      if tag in ["humidity", "all"]:
        result += f' {MESSAGES[LANG]["messages"]["humidity"]} = {data_tag["humidity"]},'
    if tag in ["wind", "weatherWind", "all"]:
      data_tag = data['wind'] if tag == 'all' and count < 2 else data
      for i, wind in enumerate(MESSAGES[LANG]["messages"]["wind"]):
        if i == 0: info = data_tag["speed"] if count < 2 else data_tag["wind_speed"]
        elif i == 1: info = data_tag["deg"] if count < 2 else data_tag["wind_deg"]
        elif i == 2: 
          if "gust" in data_tag: 
            info = data_tag["gust"] 
        elif "wind_gust" in data_tag: 
            info = data_tag["wind_gust"] 
        else: 
            continue
        result += f' {wind} = {info},'
    if tag in ["visibility","all"]:
      if cnt > 0 and 'visibility' not in data: continue 
      data_tag = data['visibility'] if tag == 'all' or count > 1 else data
      result += f' {MESSAGES[LANG]["messages"]["visibility"]} = {data_tag},'
    if tag in ["sunrise","all"]:
      data_tag = data['sys'] if tag == 'all' and count < 2 else data
      for i, wind in enumerate(MESSAGES[LANG]["messages"]["sunrise"]):
        if i == 0: info = data_tag["sunrise"]
        elif i == 1: info = data_tag["sunset"]
        result += f' {wind} = {time.strftime("%H:%M:%S", time.gmtime(info + 3 * 60 * 60))},'
  return result

In [219]:
def get_weather(dt, location, actions, count=2, LANG=LANG, DISP=False):
  #p(dt, location)
  city_w = city = location[0][0]  # assume one city (not handling if there are 2 cities)
  #p(city, city_w)
  result = ''
  # check if city in api_city list
  if city in df_CITIES_API.index.values:
    result = df_CITIES_API.loc[city]
  if len(result) == 0: # not found, get district
    if city in cities_il_df.index.values:
      city_w = cities_il_df.loc[city].district
  elif len(result) > 0: # more than one city -> need country
    pass

  #p(city_w, location)
  country = location[0][1]
  #p(country, len(countries_df.Name))
  if country in list(countries_df.Name):
    country_code = countries_df[countries_df.Name == country].index[0]
  else:
    country_code = ''
  #p(city_w, country_code)#

  fahrenheit = True if len([action for action in actions if action[0] == "fahrenheit"]) > 0 else False
  #p('in get_weather:', fahrenheit)
  city_weather_resp = call_web(city_w, country_code, fahrenheit, count)

  if DISP: p(city_weather_resp)
  if DISP: p('get_w', actions)
  if city_weather_resp is None:
    return MESSAGES[LANG]["messages"]["api_error"]
    # check what informtion the user asked for

  city_weather = city_weather_resp if count < 2 else city_weather_resp["current"]

  default = city_weather["weather"][0]["description"]
  icon = city_weather["weather"][0]["icon"]
  url_icon = f"{icon_url}".format(icon)
  if DISP: p(url_icon)
  if DISP: p('get_w', default)
  result = ''

  for cnt in range(count):
    if cnt > 0: 
      result += f'\nWeather in {cnt} days: '
      city_weather = city_weather_resp["daily"][0]
    new_result = get_weather_info(actions, count, cnt, city_weather, default, LANG, DISP)
    #p(cnt, new_result)
    result += new_result

  if result == '': result = default
  if result[-1] == ',': result = result[:-1]
  country = location[0][1]
  if LANG != 'en': # in case there are parts of the answer which are not in the proper langauge
    response = translator.translate(city, dest=LANG) # need to separet ity from country in case one of them has multi words and google doe not keep the qoutation mark as needed
    #response = translator.translate(f"({city}, {country})", dest=LANG)
    #city, country = response.text
    p(f"{response.origin} ({response.src}) --> {response.text} ({response.dest})")
    city = response.text
    response = translator.translate(country, dest=LANG)
    country = response.text
  response =  MESSAGES[LANG]["messages"]["web_msg"].format(city.capitalize(), country.capitalize(), result)
  return response
'''    
#answer = get_weather('current', [('jerusalem', 'israel')], [('clouds', 'cloud'), ('temp', 'temp'), ('pressure', 'pressure'), ('humidity', 'humidity'), ('wind', 'wind'), ('visibility', 'wind'), ('sunrise', 'wind'), ('weather','rain')])
answer = get_weather('current', [('jerusalem', 'israel')], 
  [('temp', 'heat'),
  ('weather', 'rain'),
  ('temp', 'temp'),
  ('pressure', 'pressure'),
  ('humidity', 'humidity'),
  ('wind', 'wind'),
  ('visibility', 'visibility'),
  ('sunrise', 'sunrise')])
'''
#{'date': 'current', 'action': ('weather', 'weather'), 'city': [('vi', 'sweden')]}
# if tag in ["weatherTemp", "weatherPressure", 'temp', "pressure", "humidity", "all"]:

answer = get_weather('current', [('tel aviv', 'israel')],  [('all', 'all'), ('visibility', 'visibility'), ('sunrise', 'sunrise'), ('temp', 'heat'), ('wind', 'wind'), ('weather','rain'), ('fahrenheit','fahrenheit')], DISP=False)
answer

vis 2 0 {'dt': 1625049515, 'sunrise': 1625020664, 'sunset': 1625071864, 'temp': 89.38, 'feels_like': 98.24, 'pressure': 1008, 'humidity': 60, 'dew_point': 73.67, 'uvi': 9.7, 'clouds': 0, 'visibility': 10000, 'wind_speed': 13.8, 'wind_deg': 280, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}
vis 2 1 {'dt': 1625043600, 'sunrise': 1625020664, 'sunset': 1625071864, 'moonrise': 0, 'moonset': 1625041200, 'moon_phase': 0.7, 'temp': {'day': 88.23, 'min': 78.12, 'max': 89.38, 'night': 80.92, 'eve': 85.84, 'morn': 78.24}, 'feels_like': {'day': 94.28, 'night': 84.83, 'eve': 90.79, 'morn': 79.29}, 'pressure': 1008, 'humidity': 57, 'dew_point': 71.08, 'wind_speed': 11.39, 'wind_deg': 283, 'wind_gust': 10.38, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': 0, 'pop': 0, 'uvi': 10.84}


'In Tel aviv, Israel the current weather is:  clear sky, temp = 89.38, pressure = 1008, humidity = 60, wind speed = 13.8, wind direction = 280, wind gust = 280, visibility = 10000, sunrise = 05:37:44, sunset = 19:51:04, no rain,\nWeather in 1 days:  clear sky, temp = 88.23, feels like = 94.28, temp min = 78.12, temp max = 89.38, pressure = 1008, humidity = 57, wind speed = 11.39, wind direction = 283, wind gust = 283, no rain'

# NLM

In [25]:
FAST_START = False

In [48]:
#nlp = en_core_web_md.load()
'''
 becuase it takes long time to init (about 1.5 minutes), and once a day the PythonAnywhere bring down all sites,
 for maintenance, and to prevet the first user from waiting too long, I adde a line in the nightly batch file
 rungui.bat that start the site atabout 3 AM
  start chrome http://drbarak.pythonanywhere.com/chatbot
'''
def update_nlp():
  matcher = PhraseMatcher(nlp.vocab)
  CITIES = set(CITIES_IL + CITIES_API_city)# + ACTIONS_patterns) it recognized all the actions as GPE
  #CITIES = list(CITIES)[:10]
  #p(CITIES)
  # disable other pipes while doing the traing to speed it up - went down from 1:40 to 0:40 minutes
  if not FAST_START:
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
      matcher.add("CITIES", None, *list(nlp.pipe(CITIES)))
#      matcher.add("ACTIONS", None, *list(nlp.pipe(ACTIONS_patterns)))

  # country codes causes problems because many are similar to regular words (eg. 'IN' similar to 'in')
  #matcher.add("CITIES_API_country_code", None, *list(nlp.pipe(CITIES_API_country_code)))
  # due to duplicates with country codes ('IL' = Israel, and Ilinoie), we have to check manually for states
  #matcher.add("CITIES_API_state_code", None, *list(nlp.pipe(CITIES_API_state_code)))

  # Register the Span extension attributes "is_country" and "is_city" 
  check_country = lambda span: span.text.lower().translate(remove_punct_dict) in COUNTRIES
  Span.set_extension('is_country', getter=check_country, force=True)

  check_city = lambda span: span.text.lower().translate(remove_punct_dict) in CITIES
  Span.set_extension('is_city', getter=check_city, force=True)

  check_action = lambda token: token.text.lower().translate(remove_punct_dict) in ACTIONS_patterns or \
                                token.lemma_.lower().translate(remove_punct_dict) in ACTIONS_patterns
  Token.set_extension('is_action', getter=check_action, force=True)

  pipes = nlp.pipe_names
  if 'gpe_component' in pipes:
    nlp.remove_pipe('gpe_component')

  if spacy.__version__ > '3':
    @Language.component("gpe_component")
    def gpe_component(doc):
        # Create an entity Span with the label "GPE" for all matches
        matches = matcher(doc)
        try:
          doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
          #if len(doc.ents) > 0:
          #  p(doc.ents, 'rain' in CITIES)
          #doc.ents += tuple([Span(doc, start, end, label="WACT") for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'ACTIONS'])
        except ValueError as e: #[E103] Trying to set conflicting doc.ents: '(2, 4, 'GPE')' and '(3, 4, 'GPE')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.
          if 'E103' in e.args[0]:
            return doc  # use the standard model of Spacy, hopefully it works (as in the case "weather in new york") or  other rounds will get the proper result
          raise
        return doc

    # Add the new component to the pipeline
    nlp.add_pipe('gpe_component', first=True)
  else:
    def gpe_component(doc):
        # Create an entity Span with the label "GPE" for all matches
        matches = matcher(doc)
        try:
          doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'CITIES']
#          if len(doc.ents) > 0:
#            p(doc.ents, 'rain' in CITIES)

#          doc.ents += tuple([Span(doc, start, end, label="WACT") for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'ACTIONS'])
        except ValueError as e: #[E103] Trying to set conflicting doc.ents: '(2, 4, 'GPE')' and '(3, 4, 'GPE')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.
          if 'E103' in e.args[0]:
            return doc  # use the standard model of Spacy, hopefully it works (as in the case "weather in new york") or  other rounds will get the proper result
          raise
        return doc

    # Add the new component to the pipeline
    nlp.add_pipe(gpe_component, first=True)
  #print(nlp.pipe_names)

update_nlp()
#get_parts("humidity nenita ellerbach", True)

<IPython.core.display.Javascript object>

# check city for (POS)

In [27]:
def check_gpe(entity, ents, ent_text, gpe, multiple_cities):
        token = None
        for e in ents:
          if ent_text == e[0]:
            token = e[1] 
            break
        # verify the GPE is a true GPE for cases that a city has a name that has another meaninig
        if token and token.text != token.lemma_: # a true GPE cannot be lemmatized
          return 'continue', gpe, 0

        # for example there is a city in Denemark that has a name 'rain'
        if token and len(ents) > 1 and (token.head == token or token.pos_ not in ['PROPN', 'PRON', 'NOUN']) and not entity._.is_country:
          valid_gpe = False
          for e in ents:
            if ent_text != e[0]:
              tk = e[1]
              if tk.head == token:  # the token is connected to another gpe, so it is a valid gpe (for example, (paris, canada) are connected
                valid_gpe = True
                return 'break', gpe, 0
          if not valid_gpe:
            if DEBUG: p('A:', ent_text)
            return 'continue', gpe, 0

        ent_type = 'state'
        if DEBUG: p('0:', ent_text, entity._.is_city, entity._.is_country)
        if entity._.is_city and not entity._.is_country: # 'canada' is a country and also a city in Portugal
          ent_type = 'city'
          if ent_text in df_CITIES_API.index: 
            code_country = df_CITIES_API.loc[ent_text].country
            if type(code_country) != str:
              multiple_cities = True
              # see note why not to use code_country[0] which sometimes give 'Key Error'
              # https://stackoverflow.com/questions/46153647/keyerror-0-when-accessing-value-in-pandas-series            
              ent_country = countries_df.loc[code_country.iloc[0]].Name
            else:
              ent_country = countries_df.loc[code_country].Name
          elif ent_text in CITIES_IL:
            ent_country = 'israel'
          else:
            ent_country = ''
        elif entity._.is_country or ent_text in CITIES_API_country_code:
          ent_type = 'country'

        if ent_type not in gpe:
          gpe[ent_type] = []
        if ent_type == 'city':
           gpe[ent_type].append((entity.text, ent_country))
        else:
          gpe[ent_type].append(entity.text)
        return None, gpe, multiple_cities

In [28]:
def get_city_country(multiple_cities, gpe):
  if DEBUG: p('1:', multiple_cities, gpe)
  if multiple_cities or ('city' in gpe and len(gpe['city']) > 1):
    if 'country' in gpe:
      # verify country code of the city is correct, when there are multiple cities
      for i, (city, city_country) in enumerate(gpe['city']):
        country = gpe['country'][0]
        if city_country != country:
          for code_country in df_CITIES_API.loc[city].country:
            if city_country == countries_df.loc[code_country].Name:
              # found a matching country
              gpe['city'][i] = (city, country)
              break
    else:
      check_capital = True
      if len(gpe['city']) == 2: # maybe entered a country which has city with that name (eg. Toronto, US = there is a city name 'us' in france and 'usa' in japan)
        # 'city': [('toronto', 'Canada'), ('us', 'france')]
        for i in range(1,-1,-1):
          city = gpe['city'][i][0].upper()
          if city in countries_df.index: #usually the 2nd "city" is the country
            city_country = countries_df.loc[city].Name
            j = 0 if i == 1 else 1
            gpe['city'] = [(gpe['city'][j][0], city_country)]
            gpe['country'] = city_country
            check_capital = False
            break
      if check_capital:
        # check if one of the cities is a capital or a large city, then use it
        #p(gpe, multiple_cities)
        for i, (city, city_country) in enumerate(gpe['city']):
          if city in capitals_df.index.values:
            # found a matching country
            city_country = capitals_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
          if city in largest_df.index.values:
            # found a matching country
            city_country = largest_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
  elif 'country' in gpe and 'city' in gpe:  # country with city, verify correct
      if DEBUG: p('2:', gpe)
      # verify country code of the city is correct, when user entered 
      city, city_country = gpe['city'][0]
      country = gpe['country'][0]
      if city_country != country:
        gpe['city'] = [(city, "ERROR: " + MESSAGES[LANG]["messages"]["city_error"].format(city, country))]
  elif 'country' in gpe and 'city' not in gpe:  # country without city, get the capital
    country = gpe['country'][0]
    #p(country)
    if country in CAPITALS:
      gpe['city'] = [(CAPITALS[country], country)]
  return gpe

# Get Parts (POS)

In [29]:
DEBUG = False
def get_parts(text, disp=False, verbose=True, remove_punct=True):
  global DEBUG
  if remove_punct:
    doc = nlp(text.lower().translate(remove_punct_dict))
  else:
    doc = nlp(text.lower().translate(space_punct_dict))
  date, gpe, ents = [], {'action':[]}, []
  multiple_cities = False
  if disp: p()
  for token in doc:
    if disp:
      p(f"Tokens of [{token.text}]: dep_={token.dep_}, ent_type={token.ent_type_}, head={token.head}, lemma_={token.lemma_}, pos_={token.pos_}, tag_={token.tag_}, is_action={token._.is_action}")
    if token._.is_action:  # checking for action first catch cities with the same name ('rain' is a city)
      tag = ACTIONS[token.lemma_]
      tag_exists = [t for t in gpe['action'] if t[0] == tag]
      if len(tag_exists) == 0:
        gpe['action'].append((tag, token.lemma_))
    elif token.ent_type_ == 'GPE': ents.append((token.text.lower(), token))
  for entity in doc.ents:
      if disp:
        print(f"Entities of [{entity.text}]: {entity.label_}='{spacy.explain(entity.label_)}', is_city={entity._.is_city}, is_country={entity._.is_country}")
      ent_text = entity.text.lower().translate(remove_punct_dict)
      if entity.label_ == 'DATE': date.append(ent_text)
      '''
      if entity._.is_action:  # 'rain' is both a city and weather action
        tag = ACTIONS[ent_text]
        gpe['action'].append((tag, ent_text))
      '''
      if entity.label_ == 'GPE':
        act, gpe, multiple_cities = check_gpe(entity, ents, ent_text, gpe, multiple_cities)
        if act == 'continue': continue
        if act == 'break': break
     
  gpe = get_city_country(multiple_cities, gpe)
    # remove city with same name as action ('rain') if there is another city
  if 'city' in gpe:
    org_gpe = gpe['city'].copy() # copy because the list is modified inside the loop
    for tup in org_gpe:
      city = tup[0]
      city_exists = [t for t in gpe['action'] if t[0] == city or t[1] == city]
      if len(city_exists) == 0: continue
      if len(gpe['city']) == 1:  # must leave at least one city
        # remove from action list if appears only once in the sentence
        n = [t for t in doc if t.text == city]
        if len(n) > 1: continue
        gpe['action'].remove(city_exists[0])
        continue
      gpe['city'].remove(tup)

  if len(date) == 0:
    date = 'current'
  if len(gpe['action']) == 0:
    gpe['action'] = [('all', 'weather')]
  else:
    for i, tup in enumerate(gpe['action']):
      if tup[0] == 'all':
        if i == 0: break
        gpe['action'].remove(tup)
        gpe['action'].insert(0, tup)
        break

  result = {'date':date}
  #result['date'] = date
  result.update(gpe)

  if disp:
    p(result)
  elif verbose and 'city' not in result:
    p(f'FAILED: org [{text}], {result}')
  return result

#DEBUG = False
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("Today weather in a Ra'anana")  
  get_parts("Today weather in a Raanana")
  get_parts("weather in toronto") # there is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db
  get_parts("weather in toronto, canada")
  get_parts("weather paris next 5 days")
  #get_parts("tell me how is the weather in the ra- anana in the")
  get_parts("rain in toronto")
  get_parts("weather in toronto, us")
  get_parts("weather in toronto, canada")
  get_parts("weather in jenin")
  get_parts("weather in jenin, israel")
  get_parts('haifa, israel')
  DEBUG = False
  get_parts("Wheather forecast in Tel aviv")
  get_parts("paris Canada")
  get_parts("heat in paris")
  get_parts("clouds, heat, rain in toronto")
  get_parts("clouds, heat, rain, temp, pressure, humidity, wind, visibility, sunrise in toronto")
  get_parts("clouds, heat and rain in rain")
#answer = get_weather('current', [('jerusalem', 'israel')], [('clouds', 'cloud'), ('temp', 'temp'), ('pressure', 'pressure'), ('humidity', 'humidity'), ('wind', 'wind'), ('visibility', 'wind'), ('sunrise', 'wind'), ('weather','rain')])
#answer = get_weather('current', [('jerusalem', 'israel')], [('clouds', 'cloud'), ('temp', 'temp'), ('pressure', 'pressure'), ('humidity', 'humidity'), ('wind', 'wind'), ('visibility', 'wind'), ('sunrise', 'wind'), ('weather','rain')])
#answer
get_parts("rain, clouds, weather in toronto, canada", True)


Tokens of [rain]: dep_=compound, ent_type=GPE, head=clouds, lemma_=rain, pos_=PROPN, tag_=NNP, is_action=True
Tokens of [clouds]: dep_=compound, ent_type=, head=weather, lemma_=clouds, pos_=PROPN, tag_=NNPS, is_action=True
Tokens of [weather]: dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=PROPN, tag_=NNP, is_action=True
Tokens of [in]: dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [toronto]: dep_=compound, ent_type=GPE, head=canada, lemma_=toronto, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [canada]: dep_=pobj, ent_type=GPE, head=in, lemma_=canada, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [rain]: GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [toronto]: GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [canada]: GPE='Countries, cities, states', is_city=True, is_country=True
{'date': 'current', 'action': [('all', 'weather'), ('weather', 'rain'), ('clouds', 'clouds')], 'city': [('toronto', 'canada')], 'country': ['canada']}


{'action': [('all', 'weather'), ('weather', 'rain'), ('clouds', 'clouds')],
 'city': [('toronto', 'canada')],
 'country': ['canada'],
 'date': 'current'}

In [30]:
get_parts("20/10/2021", True, remove_punct=False)
get_parts("20/10/2021", True, remove_punct=False)
get_parts("20 10 2021", True, remove_punct=False)


Tokens of [20]: dep_=ROOT, ent_type=DATE, head=20, lemma_=20, pos_=NUM, tag_=CD, is_action=False
Tokens of [10]: dep_=nummod, ent_type=DATE, head=2021, lemma_=10, pos_=NUM, tag_=CD, is_action=False
Tokens of [2021]: dep_=quantmod, ent_type=DATE, head=20, lemma_=2021, pos_=NUM, tag_=CD, is_action=False


<IPython.core.display.Javascript object>

Entities of [20 10 2021]: DATE='Absolute or relative dates or periods', is_city=False, is_country=False
{'date': ['20 10 2021'], 'action': [('all', 'weather')]}

Tokens of [20]: dep_=ROOT, ent_type=DATE, head=20, lemma_=20, pos_=NUM, tag_=CD, is_action=False
Tokens of [10]: dep_=nummod, ent_type=DATE, head=2021, lemma_=10, pos_=NUM, tag_=CD, is_action=False
Tokens of [2021]: dep_=quantmod, ent_type=DATE, head=20, lemma_=2021, pos_=NUM, tag_=CD, is_action=False


<IPython.core.display.Javascript object>

Entities of [20 10 2021]: DATE='Absolute or relative dates or periods', is_city=False, is_country=False
{'date': ['20 10 2021'], 'action': [('all', 'weather')]}

Tokens of [20]: dep_=ROOT, ent_type=DATE, head=20, lemma_=20, pos_=NUM, tag_=CD, is_action=False
Tokens of [10]: dep_=nummod, ent_type=DATE, head=2021, lemma_=10, pos_=NUM, tag_=CD, is_action=False
Tokens of [2021]: dep_=quantmod, ent_type=DATE, head=20, lemma_=2021, pos_=NUM, tag_=CD, is_action=False


<IPython.core.display.Javascript object>

Entities of [20 10 2021]: DATE='Absolute or relative dates or periods', is_city=False, is_country=False
{'date': ['20 10 2021'], 'action': [('all', 'weather')]}


{'action': [('all', 'weather')], 'date': ['20 10 2021']}

In [31]:
if TEST:
  get_parts("paris Canada")
  get_parts("las vegas, united states")
  get_parts("las vegas, US")  
  get_parts("weather in las vegas, united states")
  get_parts("weather in las vegas, US")
  get_parts("weather in new york")    
  get_parts("weather in new york us")
get_parts("paris, Canada")

{'action': [('all', 'weather')],
 'city': [('paris', 'canada')],
 'country': ['canada'],
 'date': 'current'}

In [32]:
DEBUG = False
if TEST:
  get_parts("please tell me if it will rain tonight in Rain")
  get_parts("tomorrow and today rain in spain")
  get_parts("tomorrow rain in madrid")
  get_parts("tomorrow rain in beersheba")
  get_parts("tomorrow rain in israel")
  get_parts("weather paris Canada")
  get_parts("weather paris, Canada")
  get_parts("tomorrow rain in yafo")
  

In [33]:
if TEST:
  get_parts("weather in new york")
  get_parts("weather in new york us")
  get_parts("weather in las vegas")
  get_parts("weather in las vegas, US")
  get_parts("weather in las vegas, united states")
  get_parts("weather in kas vegas")
  get_parts("weather in kas vegas, US")
  get_parts("what is the weather in summer, Israel")
# ask before switching a langauge
get_parts("weather in las vegas, us", True)
get_parts("weather in new england, us", True)

FAILED: org [weather in kas vegas], {'date': 'current', 'action': [('all', 'weather')], 'state': ['kas vegas']}
FAILED: org [weather in kas vegas, US], {'date': 'current', 'action': [('all', 'weather')], 'state': ['kas vegas']}

Tokens of [weather]: dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=NOUN, tag_=NN, is_action=True
Tokens of [in]: dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [las]: dep_=compound, ent_type=GPE, head=vegas, lemma_=las, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [vegas]: dep_=compound, ent_type=GPE, head=us, lemma_=vegas, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [us]: dep_=pobj, ent_type=GPE, head=in, lemma_=us, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [las vegas]: GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [us]: GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'action': [('all', 'weather')], 'city': [('las vegas', 'united states')], 'country': 'united states'}

Tokens of [weather]: dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=NOUN, tag_=NN, is_action=True
Tokens of [in]: dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [new]: dep_=compound, ent_type=, head=england, lemma_=new, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [england]: dep_=compound, ent_type=GPE, head=us, lemma_=england, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [us]: dep_=pobj, ent_type=GPE, head=in, lemma_=us, pos_=PROPN, tag_=NNP, is_action=False


<IPython.core.display.Javascript object>

Entities of [england]: GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

Entities of [us]: GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'action': [('all', 'weather')], 'city': [('england', 'united states')], 'country': 'united states'}


{'action': [('all', 'weather')],
 'city': [('england', 'united states')],
 'country': 'united states',
 'date': 'current'}

In [34]:
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("weather in tel aviv")
  get_parts("tomorrow weather in haifa")
  get_parts("tomorrow rain in beersheba")
  get_parts("weather paris today and tomorrow")
  get_parts("weather paris")
  get_parts("weather paris in December")
  get_parts("ra'anana")
  get_parts("Today weather in a Tel Aviv")
  get_parts("weather paris")
  get_parts("paris weather ")
  get_parts('Paris weather in')
  get_parts("weather shelomi")

In [35]:
if TEST:
  # example of successful chat although get parts failed
  get_parts("weather in tel-aviv")
  get_parts("weather in TelAviv")   # gave answer for Israel/Jerusalem
  get_parts("what is tel aviv")
  get_parts("wether in tel aviv")
  get_parts("please tell me if it will rain tonight in Rain")

FAILED: org [weather in tel-aviv], {'date': 'current', 'action': [('all', 'weather')]}
FAILED: org [weather in TelAviv], {'date': 'current', 'action': [('all', 'weather')]}


In [36]:
if TEST:
  # fixed in round1 of the bot (remove stop words and dashes followed by a space '- ')
  get_parts("weather ra- anana") # for google
  get_parts("tell me how is the weather in the ra- anana in the")

FAILED: org [weather ra- anana], {'date': 'current', 'action': [('all', 'weather')]}
FAILED: org [tell me how is the weather in the ra- anana in the], {'date': 'current', 'action': [('all', 'weather')]}


# Run the bot

In [37]:
nltk.download(['punkt', 'stopwords'])
stopwords = nltk.corpus.stopwords.words('english')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<IPython.core.display.Javascript object>

> when similarity was .075 it matched the sentence

>"tell me how is the weather in the ra- anana in the"

>to "what's up" of tag ["greeting"] so we increased it to 0.85

## Google search

In [38]:
def get_google(text):
  url = "https://google.com/search?q=" + text + '&hl=en'
  request_result = requests.get( url )
  soup = bs4.BeautifulSoup(request_result.text, "html.parser")
  spans = soup.find_all('span', class_='BNeawe')
  new_text = ''
  for span in spans: 
    if isinstance(span.contents[0], bs4.NavigableString):
      new_text += str(span.contents[0]) + ' '
  return new_text
get_google("weather ra anana")

"Ra'anana, Israel  /  Weather "

In [39]:
def add_language(lang):
  lang_exists = [t for t in MESSAGES_lang if t == lang]
  if len(lang_exists) > 0:
    p(f'{lang} already exists')
    return
  MESSAGES_lang.append(lang)
  p(MESSAGES_lang)
  MESSAGES.update({lang:{'messages':{}}})
  new_msg = list(translator.translate(list(MESSAGES_en.values()), dest=lang))
  for i, k in enumerate(MESSAGES_en.keys()):
    if type(new_msg[i]) is list:
      for msg in new_msg[i]:
        MESSAGES[lang]["messages"].update({k: msg.text})
      continue
    MESSAGES[lang]["messages"].update({k: new_msg[i].text})
#  p(MESSAGES[lang])
  new_msg = translator.translate(list(intents_en), dest=lang)
  intents.update({lang:{'messages':[]}})
  msg_arr, i = [], 0
  for dict_ in intents['en']['messages']:
    resp, msg = [], {}
    for k, v in dict_.items():
      if k == 'tag' or k == 'patterns':
        msg.update({k: v})
        continue
      for j in range(len(intents['en']["messages"][i]["responses"])):
        resp.append(new_msg[i][j].text)
    msg.update({"responses": resp})
    msg_arr.append(msg)    
    i += 1
  intents[lang].update({"messages": msg_arr})
  # save the new langauge
  new_d = {"intents": {"language": {lang: {'messages': {}}}},
          "messages": {"language": {lang: {'messages': {}}}}}
  new_d['intents']['language'][lang] = intents[lang]
  new_d['messages']['language'][lang] = MESSAGES[lang]
  # read old new langaused that were not yet added to main messages file
  list_d = []
  try:
    with open(path + 'new_messages.json', "r") as f:
      list_d = json.loads(f.read())
  except Exception as error:
    pass
    #p(error)
  list_d.insert(0, new_d) #append(new_d)
  #p(list_d)
  with open(path + 'new_messages.json', "w") as f:
    json.dump(list_d, f, indent=1, ensure_ascii = False)
    
#p(translator.detect('الشغل'))
# 'xab' is in Hmong langauge (hmn) 
#p(translator.detect('Работа'))
#add_language('iw') # hebrew
#add_language('ar') # arabic
#add_language('ru') # russian
#add_language('fr')

In [54]:
GOOGLE = True
do = True
GREETING, NOANSWER = 0, 3
#p(intents['intents'][GREETING]['responses'][0])
min_similarity = 0.84
VERBOSE = 2
LANG = 'en'
new_lang = ''

while do:
  user_msg = input().lower().strip()
  if user_msg == '':
    answers = intents[LANG]["messages"][NOANSWER]['responses']
    p("WeatherBot: " + random.choice(answers))
    continue

  if new_lang != '':
    if user_msg == '1':
      if new_lang not in MESSAGES_lang or new_lang=='fr':
        add_language(new_lang)
      LANG = new_lang
    user_msg = org_msg
    new_lang = ''
  else:
    response = translator.detect(user_msg)
    if response.lang == 'iw': response.lang = 'he'
    if response.lang != LANG:
      p(response.lang)
      if response.lang in ['en','he']:
        LANG = response.lang
      else:
        verify = True
        if response.lang not in MESSAGES_lang:
            # add langauge only when sure - of at least 3 words of length 5 letters each - to prevent erroneous detection
          sentence = ' '.join(filter(None, user_msg.split(' ')))
          if not (len(sentence) > 11 and len(user_msg.split()) > 2):
            verify = False  # False "new langauge" - google think that 'haifa israel' is in arabic, and 'Paris, frnace' is in french
        if verify:
          new_lang = response.lang
          org_msg = user_msg
          p("WeatherBot: " + MESSAGES[LANG]["messages"]["new_lang"].format(new_lang))
          continue
    elif LANG == 'en': # check if the google translate stopped working
      test = translator.detect('שלום')
      if test.lang == LANG:
        p('Note: Google translate stopped working temporarily -> multi-lingual option is not functional')

  round = 0
  answer = ''
  org_msg = user_msg
  tag = ''
  while round < 5 and do:
    if VERBOSE: p(f'round {round}: {user_msg}')
    doc1 = nlp(user_msg)
    answers, tag = None, None
    top_similarity = 0
    # verify not an erroneous finding (if we have GPE in the doc we assume it is not a general request)
    if len([ent for ent in doc1.ents if ent.label_ == 'GPE']) == 0:
      for ints in intents[LANG]["messages"]:
        #p(ints)
        for text in ints['patterns']:
          doc2 = nlp(text)
          # Get the similarity of doc1 and doc2
          if doc1.vector_norm == 0 or doc2.vector_norm == 0:
            continue
          similarity = doc1.similarity(doc2)
          if VERBOSE > 2: p(f"{similarity}, {doc2}, {[ints['tag']]}")
          if similarity > top_similarity:
            top_similarity = similarity
            answers = ints['responses']
            tag = ints['tag']
    #  p(top_similarity, tag)
    #  p(answers)
      if top_similarity > min_similarity:
        if tag == "goodbye":
          do = False
        if tag in ["goodbye", "greeting", "thanks", "options"]:
          answer = random.choice(answers)
          break
    if round == 2 and GOOGLE: round = 3 # to know we got the answer from get_parts()
    parts = get_parts(user_msg, VERBOSE > 1, False)
    if VERBOSE:  p('x:', parts)
    if 'date' in parts and 'city' in parts and 'ERROR' not in parts['city'][0][1]:
      #p(parts['city'][0][0], parts['city'])
      p('calling get_weather', LANG)
      answer = get_weather(parts['date'], parts['city'], parts['action'], LANG)
      break
    elif 'city' in parts and 'ERROR' in parts['city'][0][1]:
      answer = parts['city'][0][1]
      break
    elif round == 0:
      round = 1
      user_msg = ''
      space = ' '
      for token in doc1:
        if token.text in stopwords: continue
        text = token.text.lower().translate(remove_punct_dict)
        if text == '': continue
        if text[-1] == '-': # take care of 'ra- anana'
          user_msg += space + text[:-1]
          space = ''
        else:
          user_msg += space + text
          space = ' '
      continue
    elif round == 1:
      round = 2
      user_msg = translator.translate(org_msg) # default translation is to english - can set source/dest langauge: src="de"/ dest="he"
      if VERBOSE:  p(f"{user_msg.origin} ({user_msg.src}) --> {user_msg.text} ({user_msg.dest})")
      user_msg = user_msg.text
      if user_msg != '':
        continue
    if round == 2 or round == 3:  # need this round because in round 2 get_google('ביי') returns empty string
      round = 4
      user_msg = get_google(org_msg)
      if user_msg != '':
        continue
  if round >= 3 and not answer == '' and not 'ERROR' in answer:
    p(type(answer))
    p(answer)
    answer += f' ({MESSAGES[LANG]["messages"]["from_google"]})'
  if answer == '': answer = MESSAGES[LANG]["messages"]["city_unknown"]  
  p(f"WeatherBot: {answer}")
  #p(tag)
# weather in jenin, israel  
# 'קשאיקר ןמ ןדרשקך (weather in israel un hebrew)
# clouds and pressure in toronto
# clouds, temp, pressure, humidity in toronto
# get_parts("clouds, heat, rain, temp, pressure, humidity, wind, visibility, sunrise in toronto")
# get_parts("clouds, heat and rain in rain", True)


temp in tel aviv in Fahrenheit
round 0: temp in tel aviv in fahrenheit

Tokens of [temp]: dep_=ROOT, ent_type=, head=temp, lemma_=temp, pos_=PROPN, tag_=NNP, is_action=True
Tokens of [in]: dep_=prep, ent_type=, head=temp, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [tel]: dep_=compound, ent_type=GPE, head=aviv, lemma_=tel, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [aviv]: dep_=pobj, ent_type=GPE, head=in, lemma_=aviv, pos_=PROPN, tag_=NNP, is_action=False
Tokens of [in]: dep_=prep, ent_type=, head=temp, lemma_=in, pos_=ADP, tag_=IN, is_action=False
Tokens of [fahrenheit]: dep_=pobj, ent_type=, head=in, lemma_=fahrenheit, pos_=NOUN, tag_=NN, is_action=True


<IPython.core.display.Javascript object>

Entities of [tel aviv]: GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'action': [('temp', 'temp'), ('fahrenheit', 'fahrenheit')], 'city': [('tel aviv', 'israel')]}
x: {'date': 'current', 'action': [('temp', 'temp'), ('fahrenheit', 'fahrenheit')], 'city': [('tel aviv', 'israel')]}
calling get_weather en
in get_weather: True
WeatherBot: In Tel aviv, Israel the current weather is:  temp = 24.02, feels like = 24.46, temp min = 23.43, temp max = 29.31
bye
round 0: bye
WeatherBot: Have a nice day


# TODO:

1. DONE: display country when getting result of weather so the user knows it is the right country - if there multiple cities

2. DONE: load jsons from github - (one file too large so download it from a shared goole link on my google drive)

3. DONE: Use flask

4. DONE: if fails to understand try round 2 - removing stop words (punctuation akready done by spacy), then round 3 use google to try to fix typo mistake and only if all fails display a message to user

5. To identify if it is a question or not - must be question

Easy:
1. Wheather forecast in Tel aviv - DONE partially, what do you mean by 'forecast'

1. How hot is it in Herzliya

1. Today’s temperature in Netanya

1. Moist in Hadera

1. How may degrees tonight in Hadera 	Ofakim

1. Weekly forecast for Ramat Hashron

1. Israel desert forecast

1. Weather in Ramat gan

1.  Moist in Givataim

1. Wave height in Ashdod

1. Moist ashkelon

1. Night degrees in Eilat

10 שאלות מורכבות על מזג אויר

1. Newt 10 days forecast in Herzliya and Raanana

1. Is there a weather forecast on Herzliya

1. Would like to get weather forecast on Hadera next week

1. North Israel weather

1. Monthly average forecast for north israel

1. Monthly average forecast on Thailand

10 שאלות שגויות לבדיקה מה ניתן

1. What are the optional cities for weather forecast

1. Who is the first prime minister of Israel

1. Who where the 3 stooges

1. What is a banana

1. Who was Napoleon

1. When was photography invented

1. Are there aliens

Extras:
1. בדיקה על תוצרי שליפה במקרה של עיר שקיימת ביותר ממדינה אחת

1. 	במשלוח של עיר כמו PARIS, התקבלו תוצאות הן על צרפת והן ארצות הברית.

1. איזה פונקציונאליות נוספת ה API יודע לתת - כרגע נבחר API אחד שיודע לעבוד עם עיר בלבד.

1. להבין את שיטות העבודה של ה API (מה קורה במידה ומבקשים תחזית “מחר” - מה הוא יודע להחזיר) - לא רלבנטי 

1. עובדים עם API בסיסי

# QUESTION:

2. Are we to handle 'Ra-anana'? we handle 'Raanana'
3. DONE - There is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db (we check now if it is one of the 100 largest cities in the world)

> spacy.__version__ 
2.2.4

> get_parts('haifa, israel', True)

haifa, dep_=compound, ent_type=GPE, head=israel, lemma_=haifa, pos_=PROPN, tag_=NNP

israel, dep_=ROOT, ent_type=GPE, head=israel, lemma_=israel, pos_=PROPN, tag_=NNP

haifa, GPE='Countries, cities, states', is_city=True, is_country=False

israel, GPE='Countries, cities, states', is_city=False, is_country=True

{'date': 'current', 'city': [('haifa', 'israel')], 'country': ['israel']}

{'city': [('haifa', 'israel')], 'country': ['israel'], 'date': 'current'}

# EXAMPLES

0 You:	weather in jenin, israel

1	Bot:	ERROR: city [jenin] not found in country [israel] - try a different spelling of the city

2	You:	weather in hebron, israel

3	Bot:	In Hebron, Israel the current weather is: clear sky

4	You:	weather in jenin

5	Bot:	In Jenin, Poland the current weather is: overcast clouds

6	You:	weather in hebron

7	Bot:	In Hebron, Palestine, state of the current weather is: clear sky

8 weather isral

9 WeatherBot: In Jerusalem, Israel the current weather is: clear sky (from Google)

10 中國的天氣

In Beijing, China the current weather is: overcast clouds (from Google)

11 погода в китае
12 погода в россии

# TEST

In [41]:
s ="Tel aviv, Israel"# מזג האוויר הנוכחי הוא: שמיים בהירים (מגוגל)"
user_msg= translator.translate(s, dest='he')
p(f"{user_msg.origin} ({user_msg.src}) --> {user_msg.text} ({user_msg.dest})")

Tel aviv, Israel (en) --> תל אביב, ישראל (he)


In [42]:
p(translator.detect('Paris, Canada'))
p(translator.detect('Работа'))

Detected(lang=fr, confidence=0.7216785)
Detected(lang=['ru', 'bg'], confidence=[0.4296875, 0.37109375])
